In [10]:
from kafka.admin import KafkaAdminClient, NewTopic


admin_client = KafkaAdminClient(
    bootstrap_servers="localhost:9092", 
    client_id='tracking'
)

topic_list = [NewTopic(name="coordinates", num_partitions=1, replication_factor=1)]
admin_client.create_topics(new_topics=topic_list, validate_only=False)

TopicAlreadyExistsError: [Error 36] TopicAlreadyExistsError: Request 'CreateTopicsRequest_v3(create_topic_requests=[(topic='coordinates', num_partitions=1, replication_factor=1, replica_assignment=[], configs=[])], timeout=30000, validate_only=False)' failed with response 'CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='coordinates', error_code=36, error_message="Topic 'coordinates' already exists.")])'.

In [18]:
import json
import folium
import time
import re
import requests
from google.transit import gtfs_realtime_pb2
from json import dumps
from kafka import KafkaProducer
from kafka import KafkaConsumer

api_url = 'https://realtime.hsl.fi/realtime/vehicle-positions/v2/hsl'  # Replace with your API endpoint
KAFKA_SERVER = 'localhost:9092'
TOPIC = 'coordinates'


def fetch_coordinates():
    response = requests.get(api_url)
    if response.status_code == 200:
        # Fetch the GTFS-RT payload
        response = requests.get(api_url)
        payload = response.content

        # Create a GTFS-RT feed message
        feed = gtfs_realtime_pb2.FeedMessage()

        # Parse and decode the payload
        feed.ParseFromString(payload)

        # Create a list to hold the decoded entities
        decoded_entities = []

        # Iterate over the feed entities
        for entity in feed.entity:
            # Convert the entity to a dictionary
            if re.search(r'47/668', entity.id):
                entity_dict = {
                    'id': entity.id,
                    'vehicle': {
                        'trip': {
                            'route_id': entity.vehicle.trip.route_id,
                            'start_time': entity.vehicle.trip.start_time,
                            'start_date': entity.vehicle.trip.start_date
                            # Add other trip attributes as needed
                        },
                        'position': {
                            'latitude': entity.vehicle.position.latitude,
                            'longitude': entity.vehicle.position.longitude,
                            'bearing': entity.vehicle.position.bearing
                            # Add other trip attributes as needed
                        },
                        'position': {
                            'latitude': entity.vehicle.position.latitude,
                            'longitude': entity.vehicle.position.longitude,
                            'bearing': entity.vehicle.position.bearing,
                            'odometer': entity.vehicle.position.odometer,
                            'speed' : entity.vehicle.position.speed
                            # Add other trip attributes as needed
                        },
                        'stop_id': entity.vehicle.stop_id,
                        'current_status': entity.vehicle.current_status,
                        'timestamp': entity.vehicle.timestamp
                    }
                }


                # Append the entity dictionary to the list
                decoded_entities.append(entity_dict)

        # Convert the list of decoded entities to a JSON array
        json_array = json.dumps(entity_dict)
        # print(type(json_array))
        return json_array
    else:
        print(f"Error fetching coordinates: {response.status_code}")
        return None


def produce_kafka_messages():
    producer = KafkaProducer(bootstrap_servers=KAFKA_SERVER, 
                             api_version=(0,11,5),
                             value_serializer=lambda x: json.dumps(x).encode('utf-8'))

    while True:
        coordinates = fetch_coordinates()
        if coordinates:
            producer.send(TOPIC, value=coordinates)
            print(f"Message produced: {coordinates}")

        time.sleep(2)  # Wait for 5 seconds before fetching coordinates again

def consume_kafka_messages():
    # Kafka server and topic information
    # Create a Kafka consumer
    consumer = KafkaConsumer(TOPIC,
                             bootstrap_servers=KAFKA_SERVER,
                             api_version=(0, 11, 5),
                             value_deserializer=lambda x: json.loads(x.decode('utf-8')))

    # Consume messages indefinitely
    for message in consumer:
        coordinates = message.value
        print(f"Message consumed: {coordinates}")
        
        # Example: Access individual fields in the coordinates
        latitude = coordinates['latitude']
        longitude = coordinates['longitude']

In [19]:
producer = KafkaProducer(bootstrap_servers=KAFKA_SERVER, 
                             api_version=(0,11),
                             value_serializer=lambda x: json.dumps(x).encode('utf-8'))

In [20]:
producer.send('coordinates', value="{'latitudew':123456}")

In [21]:
metrics = producer.metrics()

# Print the logs of all sent messages
for metric_name, metric_value in metrics.items():
    print(f"{metric_name}: {metric_value}")

kafka-metrics-count: {'count': 56.0}
producer-metrics: {'connection-close-rate': 0.03254738652352894, 'connection-creation-rate': 0.06275961889221428, 'select-rate': 0.8158494943994616, 'io-wait-time-ns-avg': 71547847.52772404, 'io-wait-ratio': 0.058372275230863434, 'io-time-ns-avg': 76972.52126840445, 'io-ratio': 6.279799255947958e-05, 'connection-count': 1.0, 'bufferpool-wait-ratio': 0.0, 'batch-size-avg': 90.0, 'batch-size-max': 90.0, 'compression-rate-avg': 1.0, 'record-queue-time-avg': 0.003000974655151367, 'record-queue-time-max': 0.003000974655151367, 'produce-throttle-time-avg': 0.0, 'produce-throttle-time-max': 0.0, 'record-send-rate': 0.03254844589037807, 'records-per-request-avg': 1.0, 'byte-rate': 2.9293601301340266, 'record-retry-rate': 0.0, 'record-error-rate': 0.0, 'record-size-max': 29.0, 'record-size-avg': 29.0, 'requests-in-flight': 0.0, 'metadata-age': 0.72343310546875, 'network-io-rate': 0.18827885667664282, 'outgoing-byte-rate': 8.127370646541749, 'request-rate': 0

In [23]:
produce_kafka_messages()

Message produced: {"id": "vehicle_position_47/668", "vehicle": {"trip": {"route_id": "1016", "start_time": "07:30:00", "start_date": "20240304"}, "position": {"latitude": 60.180179595947266, "longitude": 24.958253860473633, "bearing": 219.0, "odometer": 6633.0, "speed": 8.6899995803833}, "stop_id": "1111146", "current_status": 0, "timestamp": 1709531420}}
Message produced: {"id": "vehicle_position_47/668", "vehicle": {"trip": {"route_id": "1016", "start_time": "07:30:00", "start_date": "20240304"}, "position": {"latitude": 60.17987823486328, "longitude": 24.957807540893555, "bearing": 217.0, "odometer": 6677.0, "speed": 6.170000076293945}, "stop_id": "1111146", "current_status": 0, "timestamp": 1709531426}}
Message produced: {"id": "vehicle_position_47/668", "vehicle": {"trip": {"route_id": "1016", "start_time": "07:30:00", "start_date": "20240304"}, "position": {"latitude": 60.17966842651367, "longitude": 24.95726203918457, "bearing": 242.0, "odometer": 6717.0, "speed": 5.860000133514

KeyboardInterrupt: 